[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# Academia LangChain

Bem-vindo à Academia LangChain!

## Contexto

Na LangChain, nosso objetivo é facilitar a construção de aplicações LLM. Um tipo de aplicação LLM que você pode construir é um agente. Há muito entusiasmo em torno da construção de agentes porque eles podem automatizar uma ampla gama de tarefas que antes eram impossíveis.

Na prática, porém, é incrivelmente difícil construir sistemas que executem essas tarefas de forma confiável. À medida que trabalhamos com nossos usuários para colocar agentes em produção, aprendemos que um maior controle é frequentemente necessário. Você pode precisar que um agente sempre chame uma ferramenta específica primeiro ou use prompts diferentes com base em seu estado.

Para resolver esse problema, construímos o [LangGraph](https://langchain-ai.github.io/langgraph/) — um framework para construir aplicações de agente e multi-agente. Separado do pacote LangChain, a filosofia central de design do LangGraph é ajudar os desenvolvedores a adicionar melhor precisão e controle nos fluxos de trabalho dos agentes, adequados para a complexidade dos sistemas do mundo real.

## Estrutura do Curso

O curso está estruturado como um conjunto de módulos, com cada módulo focado em um tema específico relacionado ao LangGraph. Você verá uma pasta para cada módulo, que contém uma série de notebooks. Um vídeo acompanhará cada notebook para ajudar a percorrer os conceitos, mas os notebooks também são independentes, o que significa que contêm explicações e podem ser visualizados independentemente dos vídeos. Cada pasta de módulo também contém uma pasta `studio`, que contém um conjunto de gráficos que podem ser carregados no [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), nosso IDE para construir aplicações LangGraph.

## Configuração

Antes de começar, siga as instruções no `README` para criar um ambiente e instalar as dependências.

## Modelos de chat

Neste curso, usaremos [Modelos de Chat](https://python.langchain.com/v0.2/docs/concepts/#chat-models), que fazem algumas coisas como receber uma sequência de mensagens como entradas e retornar mensagens de chat como saídas. LangChain não hospeda nenhum Modelo de Chat, em vez disso, dependemos de integrações de terceiros. [Aqui](https://python.langchain.com/v0.2/docs/integrations/chat/) está uma lista de integrações de modelos de chat de terceiros dentro do LangChain! Por padrão, o curso usará [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) porque é tanto popular quanto eficiente. Como observado, certifique-se de que você tem uma `OPENAI_API_KEY`.

Vamos verificar se sua `OPENAI_API_KEY` está configurada e, se não estiver, você será solicitado a inseri-la.

In [8]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [4]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

[Aqui](https://python.langchain.com/v0.2/docs/how_to/#chat-models) está um guia útil para todas as coisas que você pode fazer com modelos de chat, mas mostraremos alguns destaques abaixo. Se você executou `pip install -r requirements.txt` conforme indicado no README, então você instalou o pacote `langchain-openai`. Com isso, podemos instanciar nosso objeto de modelo `ChatOpenAI`. Se você está se inscrevendo na API pela primeira vez, deve receber [créditos gratuitos](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) que podem ser aplicados a qualquer um dos modelos. Você pode ver os preços para vários modelos [aqui](https://openai.com/api/pricing/). Os notebooks usarão por padrão o `gpt-4o` porque é um bom equilíbrio entre qualidade, preço e velocidade [veja mais aqui](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), mas você também pode optar pelos modelos da série `gpt-3.5` de menor preço.

Existem [alguns parâmetros padrão](https://python.langchain.com/v0.2/docs/concepts/#chat-models) que podemos definir com modelos de chat. Dois dos mais comuns são:

* `model`: o nome do modelo
* `temperature`: a temperatura de amostragem

A `temperature` controla a aleatoriedade ou criatividade da saída do modelo, onde uma temperatura baixa (próxima a 0) gera saídas mais determinísticas e focadas. Isso é bom para tarefas que requerem precisão ou respostas factuais. Alta temperatura (próxima a 1) é boa para tarefas criativas ou para gerar respostas variadas.

In [5]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

Os modelos de chat no LangChain possuem vários [métodos padrão](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). Na maior parte do tempo, usaremos:

* `stream`: transmitir pedaços da resposta de forma contínua
* `invoke`: chamar a cadeia em uma entrada

E, como mencionado, os modelos de chat recebem [mensagens](https://python.langchain.com/v0.2/docs/concepts/#messages) como entrada. As mensagens têm um papel (role) que descreve quem está enviando a mensagem e uma propriedade de conteúdo (content). Falaremos muito mais sobre isso mais tarde, mas aqui vamos apenas mostrar o básico.

In [6]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

APIConnectionError: Connection error.

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [4]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-d6f6b682-e29a-44de-b45e-79fad1e405e5-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [5]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c75d3f0f-2d71-47be-b14c-42b8dd2b4b08-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [10]:
_set_env("TAVILY_API_KEY")

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")

In [7]:
search_docs

[{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'content': 'LangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner.'},
 {'url': 'https://langchain-ai.github.io/langgraph/',
  'content': 'Overview LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures, differentiating it from DAG-based solutions. As a ...'},
 {'url': 'https://www.youtube.com/watch?v=nmDFSVRnr4Q',
  'content': 'LangGraph is an extension of LangChain enabling Multi-Agent conversation and cyclic chains. This video explains the basics of LangGraph and codesLang